In [1]:
from data import Data
import numpy as np
from matplotlib import pyplot as plt
import itertools
%load_ext autoreload
%autoreload 2

In [2]:
d = Data(first=75, x_transpose=(0, 3, 1, 2))
X_train,y_train = d.get_train()
X_cross, y_cross = d.get_dev()
X_test,y_test = d.get_test()

data/dogcat/PetImages/Cat
loaded 100


In [3]:
y_train = y_train
print(X_train.shape)
print(y_train.shape)
print(X_cross.shape)
print(y_cross.shape)
print(X_test.shape)
print(y_test.shape)

(97, 3, 128, 128)
(97,)
(30, 3, 128, 128)
(30,)
(23, 3, 128, 128)
(23,)


In [4]:
import torch
import torch.nn as nn
import torch.optim as optim

from torch.utils.data import DataLoader
from torch.utils.data import sampler
import torch.nn.functional as F
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

USE_GPU = True
if USE_GPU:
    dtypeFloat = torch.cuda.FloatTensor
    dtypeLong = torch.cuda.LongTensor
else:
    dtypeFloat = torch.FloatTensor
    dtypeLong = torch.LongTensor

In [5]:
class Flatten(nn.Module):
    def forward(self, x): 
        return flatten(x)

In [6]:
class MyCustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __getitem__(self, index):
        img = self.X[index]
        label = self.y[index]
        return (img, label)

    def __len__(self):
        return self.X.shape[0]

In [7]:
NUM_TRAIN = X_train.shape[0]
train_dataset = MyCustomDataset(X_train, y_train)
loader_train = DataLoader(train_dataset, batch_size=20,sampler=sampler.SubsetRandomSampler(range(NUM_TRAIN)))

cross_dataset = MyCustomDataset(X_cross, y_cross)
loader_cross = DataLoader(cross_dataset, batch_size=20)

In [8]:
def flatten(X):
    N = X.shape[0]
    return X.view(N,-1)

In [9]:
def check_accuracy(loader, model): 
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            scores = model(Variable(x.float().type(dtypeFloat)))
            _, preds = scores.max(1)
            num_correct += (preds == Variable(y.long().type(dtypeLong))).sum()
            num_samples += preds.size(0)
        acc = float(num_correct) / num_samples
        print('Got %d / %d correct (%.2f)' % (num_correct, num_samples, 100 * acc))

In [34]:
print_every = 200
def train(m, am, optimizer, epochs=150):
    loss_arr = []
    for e in range(epochs):
        for t, (x, y) in enumerate(loader_train):
            am.train()
            m.train()
            
            attention = am(Variable(x.float().type(dtypeFloat)))
            
            scores = m(Variable(x.float().type(dtypeFloat)) * attention)
            
            attn_mean = torch.mean(torch.mean(attention, dim=2))
            attn_var = torch.mean(torch.std(attention, dim=2) ** 2)
            
            loss = F.cross_entropy(scores, Variable(y.long().type(dtypeLong))) + 0.15 * attn_mean - 0.4 * attn_var
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if t % print_every == 0:
                print('Iteration %d, loss = %.4f, attn_mean=%.4f' % (t, loss, torch.mean(attention)))
                loss_arr += [loss.item()]
                if (t % print_every ** 2 == 0):
                    print ("train acc:")
                    check_accuracy(loader_train, m)
                    print ("cross acc:")
                    model.eval()
                    check_accuracy(loader_cross, m)
    return loss_arr

In [35]:
def imshow_noax(img, normalize=True):
    """ Tiny helper to show images as uint8 and remove axis labels """
    if normalize:
        img_max, img_min = np.max(img), np.min(img)
        img = 255.0 * (img - img_min) / (img_max - img_min)
    plt.imshow(img.astype('uint8'))
    plt.gca().axis('off')

In [ ]:
# hidden_layer_size = 500
# for i in range(20):
#     plt.subplot(4, 5, i+1)
#     print (X_train[i].transpose(1,2,0).shape)
#     imshow_noax(X_train[i].transpose(1,2,0), normalize=False)
#     print(y_train[i])

plt.show()
attn_channel_1 = 16
attn_channel_2 = 16
attn_channel_3 = 16
attn_channel_4 = 16
attn_channel_5 = 16

channel_1 = 16
channel_2 = 32
channel_3 = 50
channel_4 = 75
channel_5 = 50
channel_6 = 32
channel_7 = 16
learning_rate = 2.2e-4
num_classes = 2

# dropout??
attentionModel = nn.Sequential(
    nn.Conv2d(3, attn_channel_1, 5, stride=2, padding=2),
    nn.ReLU(),
    nn.Conv2d(attn_channel_1, attn_channel_2, 3, stride=1, padding=1),
    nn.Conv2d(attn_channel_2, attn_channel_3, 3, stride=2, padding=1),
    nn.ReLU(),
    nn.ConvTranspose2d(attn_channel_3, attn_channel_2, 3, stride=2, padding=1, output_padding=1),
    nn.ConvTranspose2d(attn_channel_2, attn_channel_1, 3, stride=1, padding=1),
    nn.ReLU(),
    nn.ConvTranspose2d(attn_channel_1, 3, 5, stride=2, padding=2, output_padding=1),
    nn.Sigmoid()
)

if USE_GPU:
    attentionModel = attentionModel.cuda()

model = nn.Sequential(
    nn.Conv2d(3, channel_1, 5, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(channel_1, channel_2, 3, padding=1),
    nn.Conv2d(channel_2, channel_3, 3, padding=1),
    torch.nn.Dropout2d(p=0.5),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(channel_3, channel_4, 3, padding=1),
    nn.Conv2d(channel_4, channel_5, 3, padding=1),
    torch.nn.Dropout2d(p=0.5),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(channel_5, channel_6, 3, padding=1),
    nn.Conv2d(channel_6, channel_7, 3, padding=1),
    torch.nn.Dropout2d(p=0.5),
    nn.MaxPool2d(2),
    Flatten(),
    nn.Linear(channel_7 * 8 * 8, num_classes)
)
if USE_GPU:
    model = model.cuda()

optimizer = optim.Adam(itertools.chain(model.parameters(), attentionModel.parameters()), lr = learning_rate)
loss_arr = train(model, attentionModel, optimizer)
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.plot(loss_arr)
plt.show()

Iteration 0, loss = 0.9334, attn_mean=0.5163
train acc:
Got 50 / 97 correct (51.55)
cross acc:
Got 14 / 30 correct (46.67)
Iteration 0, loss = 0.6467, attn_mean=0.5086
train acc:
Got 55 / 97 correct (56.70)
cross acc:
Got 19 / 30 correct (63.33)
Iteration 0, loss = 0.7084, attn_mean=0.5040
train acc:
Got 48 / 97 correct (49.48)
cross acc:
Got 14 / 30 correct (46.67)
Iteration 0, loss = 0.7198, attn_mean=0.4986
train acc:
Got 63 / 97 correct (64.95)
cross acc:
Got 20 / 30 correct (66.67)
Iteration 0, loss = 0.6487, attn_mean=0.4951
train acc:
Got 62 / 97 correct (63.92)
cross acc:
Got 19 / 30 correct (63.33)
Iteration 0, loss = 0.7281, attn_mean=0.4924
train acc:
Got 66 / 97 correct (68.04)
cross acc:
Got 20 / 30 correct (66.67)
Iteration 0, loss = 0.7154, attn_mean=0.4920
train acc:
Got 74 / 97 correct (76.29)
cross acc:
Got 19 / 30 correct (63.33)
Iteration 0, loss = 0.8048, attn_mean=0.4910
train acc:
Got 73 / 97 correct (75.26)
cross acc:
Got 19 / 30 correct (63.33)
Iteration 0, los

Iteration 0, loss = 0.0508, attn_mean=0.4082
train acc:
Got 91 / 97 correct (93.81)
cross acc:
Got 21 / 30 correct (70.00)
Iteration 0, loss = 0.2508, attn_mean=0.4054
train acc:
Got 92 / 97 correct (94.85)
cross acc:
Got 22 / 30 correct (73.33)
Iteration 0, loss = 0.2158, attn_mean=0.3996
train acc:
Got 94 / 97 correct (96.91)
cross acc:
Got 21 / 30 correct (70.00)
Iteration 0, loss = 0.1246, attn_mean=0.3954
train acc:
Got 94 / 97 correct (96.91)
cross acc:
Got 22 / 30 correct (73.33)
Iteration 0, loss = 0.1074, attn_mean=0.3995
train acc:
Got 93 / 97 correct (95.88)
cross acc:
Got 22 / 30 correct (73.33)
Iteration 0, loss = 0.0514, attn_mean=0.3956
train acc:
Got 94 / 97 correct (96.91)
cross acc:
Got 21 / 30 correct (70.00)
Iteration 0, loss = 0.3102, attn_mean=0.3948
train acc:
Got 94 / 97 correct (96.91)
cross acc:
Got 21 / 30 correct (70.00)
Iteration 0, loss = 0.3541, attn_mean=0.3886
train acc:
Got 90 / 97 correct (92.78)
cross acc:
Got 20 / 30 correct (66.67)
Iteration 0, los